In [ ]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

%matplotlib
ROOT_DIR = './'
print("Ready!")

In [ ]:
def round_to_hour(dt):
    if dt.minute > 30:#date.replace(minute = 0, hour = 0)
        if dt.hour == 23:
            return dt.replace(minute = 0, hour = 0, second = 0)
        else:
            return dt.replace(minute = 0, hour = dt.hour + 1, second = 0) 
    else:
        return dt.replace(minute = 0, second = 0)
    
def round_dt_to_10min(dt):
    try:
        tmp_min = int(str(int(dt.minute/10)) + '0') if dt.minute/10 > 1 else int(dt.minute)
    except ValueError as e:
        print(e, dt)
        return 'Null'
    if (dt.minute - tmp_min) < 5:
        return dt.replace(minute = tmp_min, second = 0)#если разница меньше 5 минут
    else: 
        try:
            if dt.hour == 23 and tmp_min == 50:
                return dt.replace(day = dt.day + 1 ,minute = 0, second = 0, hour = 0) #для 23:55+ переход на некст день
        except ValueError:
            try:
                return dt.replace(day = 1 ,minute = 0, second = 0, hour = 0, month = dt.month + 1) #для 23:55+ переход на некст месяц
            except ValueError:
                return dt.replace(day = 1 ,minute = 0, second = 0, hour = 0, month = 1, year = dt.year + 1)
        if tmp_min == 50:
            return dt.replace(minute = 0, second = 0, hour = dt.hour + 1)#для НН:55+ добавляем час
        return dt.replace(minute = tmp_min + 10, second = 0)
    
def round_to_half_hour(dt):
    #q = int(dt.minute/15)
    #print(q, dt)
    td = timedelta(hours = 1)
    if dt.minute <= 15:
        return dt.replace(minute = 0)
    if dt.minute > 15 & dt.minute < 45:
        return dt.replace(minute = 30)
    if dt.minute >= 45:
        return dt.replace(minute = 0) + td


In [ ]:
mes_df = pd.read_excel(os.path.join(ROOT_DIR, 'скд.xlsx'),sheet_name = 'скд').replace('Null', np.nan).drop(columns = [ 'wwRetrievalMode','Дата и время.1'])
mes_df['34апп.сумма.3нитка'] = mes_df['DK14_SKD_Deg_003.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_003.FRC3009'].astype(float)
mes_df['34апп.сумма.4нитка'] = mes_df['DK14_SKD_Deg_004.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_004.FR3012'].astype(float)
mes_df['34апп.сумма.5нитка'] = mes_df['DK14_SKD_Deg_005.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_005.FR3015'].astype(float)

#подача сухого полимера
mes_df['DK14_SKD_Deg_003.F1'] /=  10
mes_df['DK14_SKD_Deg_004.F1'] /= 10
mes_df['DK14_SKD_Deg_005.F1'] /= 10

# formula = 'полимер/пар'
# mes_df['34апп.удел.сумма.3нитка'] = mes_df['DK14_SKD_Deg_003.F1'].astype(float)/(mes_df['DK14_SKD_Deg_003.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_003.FRC3009'].astype(float))
# mes_df['34апп.удел.сумма.4нитка'] = mes_df['DK14_SKD_Deg_004.F1'].astype(float)/(mes_df['DK14_SKD_Deg_004.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_004.FR3012'].astype(float))
# mes_df['34апп.удел.сумма.5нитка'] = mes_df['DK14_SKD_Deg_005.F1'].astype(float)/(mes_df['DK14_SKD_Deg_005.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_005.FR3015'].astype(float))
# mes_df['удел.сумм.3нитка'] = mes_df['DK14_SKD_Deg_003.F1'].astype(float)/mes_df['Суммарная подача пара на 3 нитку, т/час'].astype(float)
# mes_df['удел.сумм.4нитка'] = mes_df['DK14_SKD_Deg_004.F1'].astype(float)/mes_df['Суммарная подача пара на 4 нитку, т/час'].astype(float)
# mes_df['удел.сумм.5нитка'] = mes_df['DK14_SKD_Deg_005.F1'].astype(float)/mes_df['Суммарная подача пара на 5 нитку, т/час'].astype(float)
# mes_df['120апп.удел.3нитка'] = mes_df['DK14_SKD_Deg_003.F1'].astype(float)/mes_df['DK14_SKD_Deg_003.F3_3'].astype(float)
# mes_df['120апп.удел.4нитка'] = mes_df['DK14_SKD_Deg_004.F1'].astype(float)/mes_df['DK14_SKD_Deg_004.F3_3'].astype(float)
# mes_df['120апп.удел.5нитка'] = mes_df['DK14_SKD_Deg_005.F1'].astype(float)/mes_df['DK14_SKD_Deg_005.F3_3'].astype(float)

#formula = 'пар/полимер, ед.изм = ГКал\ч'
# mes_df['34апп.удел.сумма.3нитка'] = ((mes_df['DK14_SKD_Deg_003.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_003.FRC3009'].astype(float))/mes_df['DK14_SKD_Deg_003.F1'].astype(float))*0.67
# mes_df['34апп.удел.сумма.4нитка'] = ((mes_df['DK14_SKD_Deg_004.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_004.FR3012'].astype(float))/mes_df['DK14_SKD_Deg_004.F1'].astype(float))*0.67
# mes_df['34апп.удел.сумма.5нитка'] = ((mes_df['DK14_SKD_Deg_005.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_005.FR3015'].astype(float))/mes_df['DK14_SKD_Deg_005.F1'].astype(float))*0.67
# mes_df['удел.сумм.3нитка'] = (mes_df['Суммарная подача пара на 3 нитку, т/час'].astype(float)/mes_df['DK14_SKD_Deg_003.F1'].astype(float))*0.67
# mes_df['удел.сумм.4нитка'] = (mes_df['Суммарная подача пара на 4 нитку, т/час'].astype(float)/mes_df['DK14_SKD_Deg_004.F1'].astype(float))*0.67
# mes_df['удел.сумм.5нитка'] = (mes_df['Суммарная подача пара на 5 нитку, т/час'].astype(float)/mes_df['DK14_SKD_Deg_005.F1'].astype(float))*0.67
# mes_df['120апп.удел.3нитка'] = (mes_df['DK14_SKD_Deg_003.F3_3'].astype(float)/mes_df['DK14_SKD_Deg_003.F1'].astype(float))*0.67
# mes_df['120апп.удел.4нитка'] = (mes_df['DK14_SKD_Deg_004.F3_3'].astype(float)/mes_df['DK14_SKD_Deg_004.F1'].astype(float))*0.67
# mes_df['120апп.удел.5нитка'] = (mes_df['DK14_SKD_Deg_005.F3_3'].astype(float)/mes_df['DK14_SKD_Deg_005.F1'].astype(float))*0.67

mes_df['Суммарная подача пара на 3 нитку, т/час'] = mes_df['DK14_SKD_Deg_003.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_003.FRC3009'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_3'].astype(float)
mes_df['Суммарная подача пара на 4 нитку, т/час'] = mes_df['DK14_SKD_Deg_004.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_004.FR3012'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_3'].astype(float)
mes_df['Суммарная подача пара на 5 нитку, т/час'] = mes_df['DK14_SKD_Deg_005.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_005.FR3015'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_3'].astype(float)

formula = 'пар/полимер, ед.изм = ГКал\т'
mes_df['34апп.удел.сумма.3нитка'] = ((mes_df['DK14_SKD_Deg_003.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_003.FRC3009'].astype(float)))*0.67/mes_df['DK14_SKD_Deg_003.F1'].astype(float)
mes_df['34апп.удел.сумма.4нитка'] = ((mes_df['DK14_SKD_Deg_004.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_004.FR3012'].astype(float)))*0.67/mes_df['DK14_SKD_Deg_004.F1'].astype(float)
mes_df['34апп.удел.сумма.5нитка'] = ((mes_df['DK14_SKD_Deg_005.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_005.FR3015'].astype(float)))*0.67/mes_df['DK14_SKD_Deg_005.F1'].astype(float)
mes_df['удел.сумм.3нитка'] = (mes_df['Суммарная подача пара на 3 нитку, т/час'].astype(float))*0.67/mes_df['DK14_SKD_Deg_003.F1'].astype(float)
mes_df['удел.сумм.4нитка'] = (mes_df['Суммарная подача пара на 4 нитку, т/час'].astype(float))*0.67/mes_df['DK14_SKD_Deg_004.F1'].astype(float)
mes_df['удел.сумм.5нитка'] = (mes_df['Суммарная подача пара на 5 нитку, т/час'].astype(float))*0.67/mes_df['DK14_SKD_Deg_005.F1'].astype(float)
mes_df['120апп.удел.3нитка'] = (mes_df['DK14_SKD_Deg_003.F3_3'].astype(float))*0.67/mes_df['DK14_SKD_Deg_003.F1'].astype(float)
mes_df['120апп.удел.4нитка'] = (mes_df['DK14_SKD_Deg_004.F3_3'].astype(float))*0.67/mes_df['DK14_SKD_Deg_004.F1'].astype(float)
mes_df['120апп.удел.5нитка'] = (mes_df['DK14_SKD_Deg_005.F3_3'].astype(float))*0.67/mes_df['DK14_SKD_Deg_005.F1'].astype(float)


# mes_df['удел.сумм.3нитка.ГКал\ч'] = mes_df['удел.сумм.3нитка']*0.67
# mes_df['удел.сумм.4нитка.ГКал\ч'] = mes_df['удел.сумм.4нитка']*0.67
# mes_df['удел.сумм.5нитка.ГКал\ч'] = mes_df['удел.сумм.5нитка']*0.67

#[['DK14_SKD_Deg_003.F3_3','DK14_SKD_Deg_004.F3_3','DK14_SKD_Deg_005.F3_3']

mes_df

In [ ]:
lims_df = pd.read_excel(os.path.join(ROOT_DIR, 'Сводный журнал результатов анализов за период ЦЗЛ ПП ПБК.xlsx'),sheet_name = 'Лист1',thousands = ',').drop(columns = ['Unnamed: 3','Unnamed: 4','Массовая доля антиоксиданта Агидол-2, Проценты относител.','Unnamed: 6','Unnamed: 8','Unnamed: 9','Unnamed: 11','Unnamed: 12', 'Unnamed: 13','Примечание','Место отбора', "Номер партии"]).rename(columns = {'Дата и время отбора':'Дата и время'}).iloc[:,:3]
lims_df['Массовая доля тримеров, %'] = (lims_df['Массовая доля тримеров, %']/1000).astype(float)
lims_df['Массовая доля углеводородов, %'] = (lims_df['Массовая доля углеводородов, %']/100).astype(float)
lims_df['Дата и время'] = lims_df['Дата и время'].astype('datetime64[ns]').apply(round_to_half_hour)

#tdelta = timedelta(minutes = 30)
#lims_df['Дата и время'] = lims_df['Дата и время'] + tdelta

fin_df = mes_df.merge(lims_df, how = 'left', on = 'Дата и время')
try:
    fin_df.to_excel('fin_df.xlsx')
except pd.FileCreateError as fce:
    print(fce)
fin_df

In [ ]:
#Для drop'a нитки
nit3 = ['DK14_SKD_Deg_003.F1',
 'DK14_SKD_Deg_003.F1_M',
 'DK14_SKD_Deg_003.F2_1',
 'DK14_SKD_Deg_003.F2_2',
 'DK14_SKD_Deg_003.F3_1',
 'DK14_SKD_Deg_003.F3_1.IsOverheat',
 'DK14_SKD_Deg_003.F3_1.T_sat',
 'DK14_SKD_Deg_003.F3_2',
 'DK14_SKD_Deg_003.F3_2.IsOverheat',
 'DK14_SKD_Deg_003.F3_2.T_sat',
 'DK14_SKD_Deg_003.F3_3',
 'DK14_SKD_Deg_003.F4',
 'DK14_SKD_Deg_003.F5',
 'DK14_SKD_Deg_003.F6',
 'DK14_SKD_Deg_003.F7_1',
 'DK14_SKD_Deg_003.F7_2',
 'DK14_SKD_Deg_003.FRC3009',
 'DK14_SKD_Deg_003.FRC3009.IsOverheat',
 'DK14_SKD_Deg_003.FRC3009.T_sat',
 'DK14_SKD_Deg_003.I1',
 'DK14_SKD_Deg_003.I2',
 'DK14_SKD_Deg_003.L1',
 'DK14_SKD_Deg_003.L1_1',
 'DK14_SKD_Deg_003.L1_2',
 'DK14_SKD_Deg_003.L2',
 'DK14_SKD_Deg_003.L2_1',
 'DK14_SKD_Deg_003.L2_2',
 'DK14_SKD_Deg_003.P1',
 'DK14_SKD_Deg_003.P2',
 'DK14_SKD_Deg_003.P2_1',
 'DK14_SKD_Deg_003.Q2',
 'DK14_SKD_Deg_003.Q2_2',
 'DK14_SKD_Deg_003.Q3_1_2',
 'DK14_SKD_Deg_003.Q3_2',
 'DK14_SKD_Deg_003.Q3_3',
 'DK14_SKD_Deg_003.Q4',
 'DK14_SKD_Deg_003.Q5',
 'DK14_SKD_Deg_003.Q6',
 'DK14_SKD_Deg_003.State1',
 'DK14_SKD_Deg_003.State2',
 'DK14_SKD_Deg_003.T1',
 'DK14_SKD_Deg_003.T2',
 'DK14_SKD_Deg_003.T3',
 'DK14_SKD_Deg_003.T4',
 'DK14_SKD_Deg_003.TT100',
 'DK14_SKD_Deg_003.TT101',
 '34апп.сумма.3нитка',
 'Суммарная подача пара на 3 нитку, т/час',
 '34апп.удел.сумма.3нитка',
 'удел.сумм.3нитка',
 '120апп.удел.3нитка',
 'DK14_Shop.REZIM_3N']


nit4 = [ 'DK14_SKD_Deg_004.F1',
 'DK14_SKD_Deg_004.F1_M',
 'DK14_SKD_Deg_004.F2_1',
 'DK14_SKD_Deg_004.F2_2',
 'DK14_SKD_Deg_004.F3_1',
 'DK14_SKD_Deg_004.F3_1.IsOverheat',
 'DK14_SKD_Deg_004.F3_1.T_sat',
 'DK14_SKD_Deg_004.F3_2',
 'DK14_SKD_Deg_004.F3_2.IsOverheat',
 'DK14_SKD_Deg_004.F3_2.T_sat',
 'DK14_SKD_Deg_004.F3_3',
 'DK14_SKD_Deg_004.F3_3.IsOverheat',
 'DK14_SKD_Deg_004.F3_3.T_sat',
 'DK14_SKD_Deg_004.F4',
 'DK14_SKD_Deg_004.F5',
 'DK14_SKD_Deg_004.F6',
 'DK14_SKD_Deg_004.F7_1',
 'DK14_SKD_Deg_004.F7_2',
 'DK14_SKD_Deg_004.FR3012',
 'DK14_SKD_Deg_004.FR3012.IsOverheat',
 'DK14_SKD_Deg_004.FR3012.T_sat',
 'DK14_SKD_Deg_004.FRC3012',
 'DK14_SKD_Deg_004.I1',
 'DK14_SKD_Deg_004.I2',
 'DK14_SKD_Deg_004.L1',
 'DK14_SKD_Deg_004.L1_1',
 'DK14_SKD_Deg_004.L1_2',
 'DK14_SKD_Deg_004.L2',
 'DK14_SKD_Deg_004.L2_1',
 'DK14_SKD_Deg_004.L2_2',
 'DK14_SKD_Deg_004.P1',
 'DK14_SKD_Deg_004.P2',
 'DK14_SKD_Deg_004.P2_1',
 'DK14_SKD_Deg_004.Q2',
 'DK14_SKD_Deg_004.Q2_2',
 'DK14_SKD_Deg_004.Q3_1_2',
 'DK14_SKD_Deg_004.Q3_2',
 'DK14_SKD_Deg_004.Q3_3',
 'DK14_SKD_Deg_004.Q4',
 'DK14_SKD_Deg_004.Q5',
 'DK14_SKD_Deg_004.Q6',
 'DK14_SKD_Deg_004.State1',
 'DK14_SKD_Deg_004.State2',
 'DK14_SKD_Deg_004.T1',
 'DK14_SKD_Deg_004.T2',
 'DK14_SKD_Deg_004.T3',
 'DK14_SKD_Deg_004.T4',
 '34апп.сумма.4нитка',
 'Суммарная подача пара на 4 нитку, т/час',
 '34апп.удел.сумма.4нитка',
 'удел.сумм.4нитка',
 '120апп.удел.4нитка']

nit5 = [ 'DK14_SKD_Deg_005.F1',
 'DK14_SKD_Deg_005.F1.8H',
 'DK14_SKD_Deg_005.F1_M',
 'DK14_SKD_Deg_005.F2_1',
 'DK14_SKD_Deg_005.F2_2',
 'DK14_SKD_Deg_005.F3_1',
 'DK14_SKD_Deg_005.F3_1.IsOverheat',
 'DK14_SKD_Deg_005.F3_1.T_sat',
 'DK14_SKD_Deg_005.F3_2',
 'DK14_SKD_Deg_005.F3_2.IsOverheat',
 'DK14_SKD_Deg_005.F3_2.T_sat',
 'DK14_SKD_Deg_005.F3_3',
 'DK14_SKD_Deg_005.F3_3.IsOverheat',
 'DK14_SKD_Deg_005.F3_3.T_sat',
 'DK14_SKD_Deg_005.F4',
 'DK14_SKD_Deg_005.F5',
 'DK14_SKD_Deg_005.F6',
 'DK14_SKD_Deg_005.F7_1',
 'DK14_SKD_Deg_005.F7_2',
 'DK14_SKD_Deg_005.FR3015',
 'DK14_SKD_Deg_005.FR3015.IsOverheat',
 'DK14_SKD_Deg_005.FR3015.T_sat',
 'DK14_SKD_Deg_005.FRC3015',
 'DK14_SKD_Deg_005.I1',
 'DK14_SKD_Deg_005.I2',#Ток мешалки?
 'DK14_SKD_Deg_005.L1',
 'DK14_SKD_Deg_005.L1_1',
 'DK14_SKD_Deg_005.L1_2',
 'DK14_SKD_Deg_005.L2',
 'DK14_SKD_Deg_005.L2_1',
 'DK14_SKD_Deg_005.L2_2',
 'DK14_SKD_Deg_005.P1',
 'DK14_SKD_Deg_005.P2',
 'DK14_SKD_Deg_005.P2_1',
 'DK14_SKD_Deg_005.Q2',
 'DK14_SKD_Deg_005.Q2_2',
 'DK14_SKD_Deg_005.Q3_1_2',
 'DK14_SKD_Deg_005.Q3_2',
 'DK14_SKD_Deg_005.Q3_3',
 'DK14_SKD_Deg_005.Q4',
 'DK14_SKD_Deg_005.Q5',
 'DK14_SKD_Deg_005.Q6',
 'DK14_SKD_Deg_005.State1',
 'DK14_SKD_Deg_005.State2',
 'DK14_SKD_Deg_005.T1',
 'DK14_SKD_Deg_005.T2',
 'DK14_SKD_Deg_005.T3',
 'DK14_SKD_Deg_005.T4',
 'DK14_SKD_Deg_005.TR103_1',
 'DK14_SKD_Deg_005.TR103_2',
 '34апп.сумма.5нитка',
 'Суммарная подача пара на 5 нитку, т/час',
 '34апп.удел.сумма.5нитка',
 'удел.сумм.5нитка',
 '120апп.удел.5нитка']
print(len(nit5), len(nit4), len(nit3))

In [ ]:
#test
list_5 = ['DK14_SKD_Deg_005.L1_1','DK14_SKD_Deg_005.L1_2',
            'DK14_SKD_Deg_005.L2_1','DK14_SKD_Deg_005.L2_2',
            'DK14_SKD_Deg_005.Q2_2',
            'DK14_SKD_Deg_005.Q3_1_2','DK14_SKD_Deg_005.Q3_2',
            'DK14_SKD_Deg_005.Q3_3','DK14_SKD_Deg_005.Q4',
            'DK14_SKD_Deg_005.F1_M','DK14_SKD_Deg_005.F2_2',
            'DK14_SKD_Deg_005.F3_1.IsOverheat','DK14_SKD_Deg_005.F3_2.IsOverheat',
            'DK14_SKD_Deg_005.F4', 'DK14_SKD_Deg_005.F5',
            'DK14_SKD_Deg_005.F6', 'DK14_SKD_Deg_005.FR3015.IsOverheat',
            'DK14_SKD_Deg_005.I1', 'DK14_SKD_Deg_005.Q2',
            'DK14_SKD_Deg_005.Q5', 'DK14_SKD_Deg_005.Q6',
            'DK14_SKD_Deg_005.State1', 'DK14_SKD_Deg_005.State2',
            'DK14_SKD_Deg_005.TR103_1','DK14_SKD_Deg_005.TR103_2',
            'DK14_SKD_Deg_005.F7_1','DK14_SKD_Deg_005.FRC3015', 'DK14_SKD_Deg_005.T3',# 57.9%, 100.0%, 57.9% missing values
            'DK14_SKD_Deg_005.F3_1.T_sat', 'DK14_SKD_Deg_005.F3_2.T_sat', 'DK14_SKD_Deg_005.FR3015.T_sat'# 6.9% missing values
         ]


list_4 = ['DK14_SKD_Deg_004.L1_1','DK14_SKD_Deg_004.L1_2','DK14_SKD_Deg_004.L2_1','DK14_SKD_Deg_004.L2_2',
          'DK14_SKD_Deg_004.Q2_2','DK14_SKD_Deg_004.Q3_1_2','DK14_SKD_Deg_004.Q3_2','DK14_SKD_Deg_004.Q3_3',
          'DK14_SKD_Deg_004.Q4','DK14_SKD_Deg_004.F7_1','DK14_SKD_Deg_004.F7_2',
          'DK14_SKD_Deg_004.F1_M','DK14_SKD_Deg_004.F2_2','DK14_SKD_Deg_004.F3_1.IsOverheat',
          'DK14_SKD_Deg_004.F3_2.IsOverheat','DK14_SKD_Deg_004.F3_3.IsOverheat','DK14_SKD_Deg_004.F4',
          'DK14_SKD_Deg_004.F5','DK14_SKD_Deg_004.F6','DK14_SKD_Deg_004.FR3012.IsOverheat',
          'DK14_SKD_Deg_004.FRC3012','DK14_SKD_Deg_004.I1','DK14_SKD_Deg_004.Q2',
          'DK14_SKD_Deg_004.Q5','DK14_SKD_Deg_004.Q6','DK14_SKD_Deg_004.State1',
          'DK14_SKD_Deg_004.State2',
          'DK14_SKD_Deg_004.F3_1.T_sat', 'DK14_SKD_Deg_004.F3_2.T_sat', 'DK14_SKD_Deg_004.FR3012.T_sat'#6.9%  missing values
         ]

list_3 = ['DK14_SKD_Deg_003.L1_1','DK14_SKD_Deg_003.L1_2',
          'DK14_SKD_Deg_003.L2_1','DK14_SKD_Deg_003.L2_2',
          'DK14_SKD_Deg_003.Q2_2', 'DK14_SKD_Deg_003.I1',
          'DK14_SKD_Deg_003.Q3_1_2','DK14_SKD_Deg_003.Q3_2',
          'DK14_SKD_Deg_003.Q3_3','DK14_SKD_Deg_003.Q4',
          'DK14_SKD_Deg_003.F7_1','DK14_SKD_Deg_003.F7_2',
          'DK14_Shop.REZIM_3N','DK14_SKD_Deg_003.F1_M',
          'DK14_SKD_Deg_003.F2_2','DK14_SKD_Deg_003.F3_1.IsOverheat',
          'DK14_SKD_Deg_003.F3_2.IsOverheat','DK14_SKD_Deg_003.F4',
          'DK14_SKD_Deg_003.F5','DK14_SKD_Deg_003.F6',
          'DK14_SKD_Deg_003.FRC3009.IsOverheat','DK14_SKD_Deg_003.Q2',
          'DK14_SKD_Deg_003.Q5','DK14_SKD_Deg_003.Q6',
          'DK14_SKD_Deg_003.State1','DK14_SKD_Deg_003.State2',
          'DK14_SKD_Deg_003.TT100','DK14_SKD_Deg_003.TT101',
          'DK14_SKD_Deg_003.T3', 'DK14_SKD_Deg_003.T4']#72.2%, 55.1%  missing values




list_tmp_4_5 = list_4.copy()
list_tmp_4_5.extend(list_5)
list_tmp_4_5.extend(nit3)
list_tmp_4_5.append('index')
print('list_tmp_4_5', len(list_tmp_4_5))
df_4_5 = fin_df[(fin_df['DK14_SKD_Deg_005.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_004.F1'] > 0.1)\
              & (fin_df['Суммарная подача пара на 5 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.5нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.5нитка'] < 8)\
              & (fin_df['удел.сумм.5нитка'] < 10)\
              & (fin_df['Суммарная подача пара на 4 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.4нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.4нитка'] < 8)\
              & (fin_df['удел.сумм.4нитка'] < 10)\
              ].reset_index().drop(columns = list_tmp_4_5)
#pandas_profiling.ProfileReport(df = df_4_5, minimal = True).to_file('df_4_5.html')#to_widgets()  

list_tmp_3_5 = list_3.copy()
list_tmp_3_5.extend(list_5)
list_tmp_3_5.extend(nit4)
list_tmp_3_5.append('index')
print('list_tmp_3_5',len(list_tmp_3_5))
df_3_5 = fin_df[(fin_df['DK14_SKD_Deg_005.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_003.F1'] > 0.1)\
              & (fin_df['Суммарная подача пара на 5 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.5нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.5нитка'] < 8)\
              & (fin_df['удел.сумм.5нитка'] < 10)\
              & (fin_df['Суммарная подача пара на 3 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.3нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.3нитка'] < 8)\
              & (fin_df['удел.сумм.3нитка'] < 10)\
              & (fin_df['DK14_Shop.REZIM_3N'] == 0)\
              ].reset_index().drop(columns = list_tmp_3_5)
#pandas_profiling.ProfileReport(df = df_3_5, minimal = True).to_file('df_3_5.html')#to_widgets()  

list_tmp_3_4 = list_4.copy()
list_tmp_3_4.extend(list_3)
list_tmp_3_4.extend(nit5)
list_tmp_3_4.append('index')
print('list_tmp_3_4',len(list_tmp_3_4))
df_3_4  = fin_df[(fin_df['DK14_SKD_Deg_004.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_003.F1'] > 0.1)\
              & (fin_df['Суммарная подача пара на 4 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.4нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.4нитка'] < 8)\
              & (fin_df['удел.сумм.4нитка'] < 10)
              & (fin_df['Суммарная подача пара на 3 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.3нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.3нитка'] < 8)\
              & (fin_df['удел.сумм.3нитка'] < 10)\
              & (fin_df['DK14_Shop.REZIM_3N'] == 0)\
              & (fin_df['DK14_SKD_Deg_004.T1'] > 50)\
              ].reset_index().drop(columns = list_tmp_3_4)
#pandas_profiling.ProfileReport(df = df_3_4, minimal = True).to_file('df_3_4.html')#to_widgets()  

list_tmp_3_4_5 = list_tmp_4_5.copy()
list_tmp_3_4_5.extend(list_3)
print('list_tmp_3_4_5',len(list_tmp_3_4_5))
df_3_4_5  = fin_df[(fin_df['DK14_SKD_Deg_004.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_003.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_005.F1'] > 0.1)\
              & (fin_df['Суммарная подача пара на 4 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.4нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.4нитка'] < 8)\
              & (fin_df['удел.сумм.4нитка'] < 10)
              & (fin_df['Суммарная подача пара на 3 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.3нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.3нитка'] < 8)\
              & (fin_df['удел.сумм.3нитка'] < 10)\
              & (fin_df['DK14_Shop.REZIM_3N'] == 0)\
              & (fin_df['Суммарная подача пара на 5 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.5нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.5нитка'] < 8)\
              & (fin_df['удел.сумм.5нитка'] < 10)\
              ].reset_index().drop(columns = list_tmp_3_4_5)
#pandas_profiling.ProfileReport(df = df_3_4_5, minimal = True).to_file('df_3_4_5.html')#to_widgets()  
print('df_3_4:',df_3_4.shape)
print('df_3_5:',df_3_5.shape)
print('df_4_5:',df_4_5.shape)
print('df_3_4_5:',df_3_4_5.shape)

In [ ]:
#.drop(columns = ['Массовая доля тримеров, %']).dropna()

In [ ]:
#1136  - всего анализов на органику
#528 от df_4_5
#149 от df_3_4
#184 от 3_5
#Подготовка df
number_of_organic_tests = fin_df[np.isnan(fin_df['Массовая доля углеводородов, %']) == False].shape[0]
dfs = []
dfs.append(df_4_5[np.isnan(df_4_5['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
dfs.append(df_3_4[np.isnan(df_3_4['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
dfs.append(df_3_5[np.isnan(df_3_5['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
losing_of_tests = number_of_organic_tests
for i in dfs:
    losing_of_tests -= i.shape[0]
print('Потеряно анализов на органику:', losing_of_tests)

In [ ]:
#Данные для обучения и теста модели
#При объеме данных менее 400 наблюдений
#линейная регрессия не способна обучиться чему-либо
tmp_df = dfs[0]
print('Размер данных=', tmp_df.shape)
X = tmp_df.drop(columns = ['Массовая доля углеводородов, %'])
y = tmp_df['Массовая доля углеводородов, %']
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), random_state=2, test_size = 0.2, train_size = 0.8)
print('Обучающие датасеты', X_train, y_train)
print('Размер Х_train', len(X_train))
print('Размер Х_test', len(X_test))

In [ ]:
# Коэффициент детерминации R-квадрат для модели с константой принимает значения от 0 до 1.
# Чем ближе значение коэффициента к 1, тем сильнее зависимость.
# При оценке регрессионных моделей это интерпретируется как соответствие модели данным.
# Для приемлемых моделей предполагается, что коэффициент детерминации должен быть хотя бы не меньше 50 %
# (в этом случае коэффициент множественной корреляции превышает по модулю 70 %). 
# Модели с коэффициентом детерминации выше 80 % можно признать достаточно хорошими (коэффициент корреляции превышает 90 %).
# Значение коэффициента детерминации 1 означает функциональную зависимость между переменными.
print('Линейная регрессия, метод наименьших квадратов:')
lr = LinearRegression().fit(X_train, y_train)
print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))
print("R-квадрат на обучающем наборе: {:.2f}".format(lr.score(X_train, y_train)))
print("R-квадрат на тестовом наборе: {:.2f}".format(lr.score(X_test, y_test)))


In [ ]:
#Вес тегов в линейной регрессии
cols = X.columns.to_list() 
coefs = lasso.coef_.tolist()
d = {}
for i in range(len(coefs)):
    d.update({cols[i]:coefs[i]})
d

In [ ]:
#Гребневая регрессия(L2 регуляризация[alpha])
#Оптимальное значение alpha зависит от конкретного используемого набора данных. 
#Увеличение alpha заставляет коэффициенты сжиматься до близких к нулю значений,
#что снижает качество работы модели на обучающем наборе, но может улучшить ее обобщающую способность.
print('Гребневая регрессия(L2 регуляризация[alpha])')
for alpha in [0, 0.0001, 0.0005,0.001, 0.01, 0.02, 0.005, 0.0075, 0.1, 0.3, 0.5, 1, 3, 10, 15, 50, 100]:
    ridge = Ridge(alpha = alpha).fit(X_train, y_train)
    print('alpha =', alpha )
    print("R-квадрат на обучающем наборе: {:.2f}".format(ridge.score(X_train, y_train)))
    print("R-квадрат на тестовом наборе: {:.2f}".format(ridge.score(X_test, y_test)))


In [ ]:
#Лассо (L1 регуляризация[alpha])
#Результат L1 регуляризации заключается в том,
#что при использовании лассо некоторые коэффициенты становятся равны точно нулю.
for alpha in [0, 0.0001, 0.0005,0.001, 0.01, 0.02, 0.005, 0.0075, 0.1, 0.3, 0.5, 1, 3, 10, 15, 50, 100]:
    lasso = Lasso(alpha = alpha).fit(X_train, y_train)
    print("R-квадрат на обучающем наборе: {:.2f}".format(lasso.score(X_train, y_train)))
    print("R-квадрат на контрольном наборе: {:.2f}".format(lasso.score(X_test, y_test)))
    print("Количество использованных признаков: {}".format(np.sum(lasso.coef_ != 0)))
    print('################################################')
#print(lasso.coef_)

In [ ]:
#Наивные байесовские классификаторы
# for alpha in [0, 0.0001, 0.0005,0.001, 0.01, 0.02, 0.005, 0.0075, 0.1, 0.3, 0.5, 1, 3, 10, 15, 50, 100]:
#     nb = MultinomialNB(alpha = alpha).fit(X_train, y_train)
#     print("R-квадрат на обучающем наборе: {:.2f}".format(nb.score(X_train, y_train)))
#     print("R-квадрат на контрольном наборе: {:.2f}".format(nb.score(X_test, y_test)))

In [ ]:
#depricated
#5nitka
#nit5_prof = pandas_profiling.ProfileReport(df = nit5, minimal = True)
#nit5_prof.to_file("5nitka_report.html")

nit5_df = nit5[(nit5['DK14_SKD_Deg_005.F1'] > 0.1)\
              #& (nit5['DK14_SKD_Deg_005.F2_1'] > 0.1)\
              #& (nit5['DK14_SKD_Deg_005.F3_3'] > 0.5)\
#               & (nit5['DK14_SKD_Deg_005.L1'] > 10)\
#               & (nit5['DK14_SKD_Deg_005.L2'] > 3)\
#               & (nit5['DK14_SKD_Deg_005.P1'] > 0.01)\
#               & (nit5['DK14_SKD_Deg_005.P2'] > 0.01)\
#               & (nit5['DK14_SKD_Deg_005.T1'] > 45)\
#               & (nit5['DK14_SKD_Deg_005.T2'] > 45)\
              & (nit5['Суммарная подача пара на 5 нитку, т/час'] > 5)\
              & (nit5['120апп.удел.5нитка'] < 5)\
              & (nit5['34апп.удел.сумма.5нитка'] < 8)\
              & (nit5['удел.сумм.5нитка'] < 10)].reset_index().drop(columns = ['DK14_SKD_Deg_005.L1_1','DK14_SKD_Deg_005.L1_2',\
                                                                    'DK14_SKD_Deg_005.L2_1','DK14_SKD_Deg_005.L2_2',\
                                                                    'DK14_SKD_Deg_005.Q2_2', 'index',\
                                                                    'DK14_SKD_Deg_005.Q3_1_2','DK14_SKD_Deg_005.Q3_2',\
                                                                    'DK14_SKD_Deg_005.Q3_3','DK14_SKD_Deg_005.Q4'])
nit5_df.info()
#pandas_profiling.ProfileReport(df = nit5_df).to_file("5nitka_report(clean).html")#to_widgets()

In [ ]:
#depricated
#4 нитка
#nit4_prof = pandas_profiling.ProfileReport(df = nit4, minimal = True)
#nit4_prof.to_file("4nitka_report.html")
nit4_df = nit4[(nit4['DK14_SKD_Deg_004.F1'] > 0.1)\
              #& (nit4['DK14_SKD_Deg_004.F2_1'] > 0.1)\
              #& (nit4['DK14_SKD_Deg_004.F3_3'] > 0.5)\
              & (nit4['DK14_SKD_Deg_004.L1'] > 10)\
              & (nit4['DK14_SKD_Deg_004.L2'] > 3)\
              & (nit4['DK14_SKD_Deg_004.P1'] > 0.01)\
              & (nit4['DK14_SKD_Deg_004.P2'] > 0.01)\
              & (nit4['DK14_SKD_Deg_004.T1'] > 45)\
              & (nit4['DK14_SKD_Deg_004.T2'] > 45)\
              & (nit4['Суммарная подача пара на 4 нитку, т/час'] > 3)\
              & (nit4['120апп.удел.4нитка'] < 5)\
              & (nit4['34апп.удел.сумма.4нитка'] < 8)\
              & (nit4['удел.сумм.4нитка'] < 10)].reset_index().drop(columns = ['DK14_SKD_Deg_004.L1_1','DK14_SKD_Deg_004.L1_2',\
                                                                    'DK14_SKD_Deg_004.L2_1','DK14_SKD_Deg_004.L2_2',\
                                                                    'DK14_SKD_Deg_004.Q2_2', 'index',\
                                                                    'DK14_SKD_Deg_004.Q3_1_2','DK14_SKD_Deg_004.Q3_2',\
                                                                    'DK14_SKD_Deg_004.Q3_3','DK14_SKD_Deg_004.Q4',\
                                                                    'DK14_SKD_Deg_004.F7_1','DK14_SKD_Deg_004.F7_2'])
nit4_df.info()
#pandas_profiling.ProfileReport(df = nit4_df).to_file("4nitka_report(clean).html")#to_widgets()


In [ ]:
#depricated
#3 нитка
#nit3_prof = pandas_profiling.ProfileReport(df = nit3, minimal = True)
#nit3_prof.to_file("3nitka_report.html")


nit3_df = nit3[(nit3['DK14_SKD_Deg_003.F1'] > 0.1)\
              #& (nit3['DK14_SKD_Deg_003.F2_1'] > 0.1)\
              #& (nit3['DK14_SKD_Deg_003.F3_3'] > 0.5)\
              & (nit3['DK14_SKD_Deg_003.L1'] > 10)\
              & (nit3['DK14_SKD_Deg_003.L2'] > 3)\
              & (nit3['DK14_SKD_Deg_003.P1'] > 0.01)\
              & (nit3['DK14_SKD_Deg_003.P2'] > 0.01)\
              & (nit3['DK14_SKD_Deg_003.T1'] > 45)\
              & (nit3['DK14_SKD_Deg_003.T2'] > 45)\
              & (nit3['Суммарная подача пара на 3 нитку, т/час'] > 3)\
              & (nit3['120апп.удел.3нитка'] < 5)\
              & (nit3['34апп.удел.сумма.3нитка'] < 8)\
              & (nit3['удел.сумм.3нитка'] < 10)\
              & (nit3['DK14_Shop.REZIM_3N'] == 0)].reset_index().drop(columns = ['DK14_SKD_Deg_003.L1_1','DK14_SKD_Deg_003.L1_2',\
                                                                    'DK14_SKD_Deg_003.L2_1','DK14_SKD_Deg_003.L2_2',\
                                                                    'DK14_SKD_Deg_003.Q2_2', 'index',\
                                                                    'DK14_SKD_Deg_003.Q3_1_2','DK14_SKD_Deg_003.Q3_2',\
                                                                    'DK14_SKD_Deg_003.Q3_3','DK14_SKD_Deg_003.Q4',\
                                                                    'DK14_SKD_Deg_003.F7_1','DK14_SKD_Deg_003.F7_2',\
                                                                    'DK14_Shop.REZIM_3N'])
nit3_df.info()
#pandas_profiling.ProfileReport(df = nit3_df).to_file("3nitka_report(clean).html")#to_widgets()

In [ ]:
#depricated
#Для работы с нитками
nit3 = fin_df[['DK14_SKD_Deg_003.F1',
 #'DK14_SKD_Deg_003.F1_M',
 'DK14_SKD_Deg_003.F2_1',
 #'DK14_SKD_Deg_003.F2_2',
 'DK14_SKD_Deg_003.F3_1',
 #'DK14_SKD_Deg_003.F3_1.IsOverheat',
 'DK14_SKD_Deg_003.F3_1.T_sat',
 'DK14_SKD_Deg_003.F3_2',
 #'DK14_SKD_Deg_003.F3_2.IsOverheat',
 'DK14_SKD_Deg_003.F3_2.T_sat',
 'DK14_SKD_Deg_003.F3_3',
 #'DK14_SKD_Deg_003.F4',
 #'DK14_SKD_Deg_003.F5',
 #'DK14_SKD_Deg_003.F6',
 'DK14_SKD_Deg_003.F7_1',
 'DK14_SKD_Deg_003.F7_2',
 'DK14_SKD_Deg_003.FRC3009',
 #'DK14_SKD_Deg_003.FRC3009.IsOverheat',
 'DK14_SKD_Deg_003.FRC3009.T_sat',
 #'DK14_SKD_Deg_003.I1',
 'DK14_SKD_Deg_003.I2',
 'DK14_SKD_Deg_003.L1',
 'DK14_SKD_Deg_003.L1_1',
 'DK14_SKD_Deg_003.L1_2',
 'DK14_SKD_Deg_003.L2',
 'DK14_SKD_Deg_003.L2_1',
 'DK14_SKD_Deg_003.L2_2',
 'DK14_SKD_Deg_003.P1',
 'DK14_SKD_Deg_003.P2',
 'DK14_SKD_Deg_003.P2_1',
 #'DK14_SKD_Deg_003.Q2',
 'DK14_SKD_Deg_003.Q2_2',
 'DK14_SKD_Deg_003.Q3_1_2',
 'DK14_SKD_Deg_003.Q3_2',
 'DK14_SKD_Deg_003.Q3_3',
 'DK14_SKD_Deg_003.Q4',
 #'DK14_SKD_Deg_003.Q5',
 #'DK14_SKD_Deg_003.Q6',
 #'DK14_SKD_Deg_003.State1',
 #'DK14_SKD_Deg_003.State2',
 'DK14_SKD_Deg_003.T1',
 'DK14_SKD_Deg_003.T2',
 'DK14_SKD_Deg_003.T3',
 'DK14_SKD_Deg_003.T4',
 #'DK14_SKD_Deg_003.TT100',
 #'DK14_SKD_Deg_003.TT101',
 'Дата и время',
 '34апп.сумма.3нитка',
 'Суммарная подача пара на 3 нитку, т/час',
 '34апп.удел.сумма.3нитка',
 'удел.сумм.3нитка',
 '120апп.удел.3нитка',
 'Массовая доля углеводородов, %',
 'Массовая доля тримеров, %','DK14_Shop.REZIM_3N']]


nit4 = fin_df[[ 'DK14_SKD_Deg_004.F1',
# 'DK14_SKD_Deg_004.F1_M',
 'DK14_SKD_Deg_004.F2_1',
 #'DK14_SKD_Deg_004.F2_2',
 'DK14_SKD_Deg_004.F3_1',
 #'DK14_SKD_Deg_004.F3_1.IsOverheat',
 'DK14_SKD_Deg_004.F3_1.T_sat',
 'DK14_SKD_Deg_004.F3_2',
 #'DK14_SKD_Deg_004.F3_2.IsOverheat',
 'DK14_SKD_Deg_004.F3_2.T_sat',
 'DK14_SKD_Deg_004.F3_3',
 #'DK14_SKD_Deg_004.F3_3.IsOverheat',
 'DK14_SKD_Deg_004.F3_3.T_sat',
 #'DK14_SKD_Deg_004.F4',
 #'DK14_SKD_Deg_004.F5',
 #'DK14_SKD_Deg_004.F6',
 'DK14_SKD_Deg_004.F7_1',
 'DK14_SKD_Deg_004.F7_2',
 'DK14_SKD_Deg_004.FR3012',
 #'DK14_SKD_Deg_004.FR3012.IsOverheat',
 'DK14_SKD_Deg_004.FR3012.T_sat',
 #'DK14_SKD_Deg_004.FRC3012',
 #'DK14_SKD_Deg_004.I1',
 'DK14_SKD_Deg_004.I2',
 'DK14_SKD_Deg_004.L1',
 'DK14_SKD_Deg_004.L1_1',
 'DK14_SKD_Deg_004.L1_2',
 'DK14_SKD_Deg_004.L2',
 'DK14_SKD_Deg_004.L2_1',
 'DK14_SKD_Deg_004.L2_2',
 'DK14_SKD_Deg_004.P1',
 'DK14_SKD_Deg_004.P2',
 'DK14_SKD_Deg_004.P2_1',
 #'DK14_SKD_Deg_004.Q2',
 'DK14_SKD_Deg_004.Q2_2',
 'DK14_SKD_Deg_004.Q3_1_2',
 'DK14_SKD_Deg_004.Q3_2',
 'DK14_SKD_Deg_004.Q3_3',
 'DK14_SKD_Deg_004.Q4',
 #'DK14_SKD_Deg_004.Q5',
 #'DK14_SKD_Deg_004.Q6',
 #'DK14_SKD_Deg_004.State1',
 #'DK14_SKD_Deg_004.State2',
 'DK14_SKD_Deg_004.T1',
 'DK14_SKD_Deg_004.T2',
 'DK14_SKD_Deg_004.T3',
 'DK14_SKD_Deg_004.T4',
 'Дата и время',
 '34апп.сумма.4нитка',
 'Суммарная подача пара на 4 нитку, т/час',
 '34апп.удел.сумма.4нитка',
 'удел.сумм.4нитка',
 '120апп.удел.4нитка',
 'Массовая доля углеводородов, %',
 'Массовая доля тримеров, %']]

nit5 = fin_df[[ 'DK14_SKD_Deg_005.F1',
 #'DK14_SKD_Deg_005.F1.8H',
 #'DK14_SKD_Deg_005.F1_M',
 'DK14_SKD_Deg_005.F2_1',
 #'DK14_SKD_Deg_005.F2_2',
 'DK14_SKD_Deg_005.F3_1',
 #'DK14_SKD_Deg_005.F3_1.IsOverheat',
 'DK14_SKD_Deg_005.F3_1.T_sat',
 'DK14_SKD_Deg_005.F3_2',
 #'DK14_SKD_Deg_005.F3_2.IsOverheat',
 'DK14_SKD_Deg_005.F3_2.T_sat',
 'DK14_SKD_Deg_005.F3_3',
 #'DK14_SKD_Deg_005.F3_3.IsOverheat',
 'DK14_SKD_Deg_005.F3_3.T_sat',
 #'DK14_SKD_Deg_005.F4',
 #'DK14_SKD_Deg_005.F5',
 #'DK14_SKD_Deg_005.F6',
 'DK14_SKD_Deg_005.F7_1',
 'DK14_SKD_Deg_005.F7_2',
 'DK14_SKD_Deg_005.FR3015',
 #'DK14_SKD_Deg_005.FR3015.IsOverheat',
 'DK14_SKD_Deg_005.FR3015.T_sat',
 #'DK14_SKD_Deg_005.FRC3015',
 #'DK14_SKD_Deg_005.I1',
 'DK14_SKD_Deg_005.I2',#Ток мешалки?
 'DK14_SKD_Deg_005.L1',
 'DK14_SKD_Deg_005.L1_1',
 'DK14_SKD_Deg_005.L1_2',
 'DK14_SKD_Deg_005.L2',
 'DK14_SKD_Deg_005.L2_1',
 'DK14_SKD_Deg_005.L2_2',
 'DK14_SKD_Deg_005.P1',
 'DK14_SKD_Deg_005.P2',
 'DK14_SKD_Deg_005.P2_1',
 #'DK14_SKD_Deg_005.Q2',
 'DK14_SKD_Deg_005.Q2_2',
 'DK14_SKD_Deg_005.Q3_1_2',
 'DK14_SKD_Deg_005.Q3_2',
 'DK14_SKD_Deg_005.Q3_3',
 'DK14_SKD_Deg_005.Q4',
 #'DK14_SKD_Deg_005.Q5',
 #'DK14_SKD_Deg_005.Q6',
 #'DK14_SKD_Deg_005.State1',
 #'DK14_SKD_Deg_005.State2',
 'DK14_SKD_Deg_005.T1',
 'DK14_SKD_Deg_005.T2',
 'DK14_SKD_Deg_005.T3',
 'DK14_SKD_Deg_005.T4',
 #'DK14_SKD_Deg_005.TR103_1',
 #'DK14_SKD_Deg_005.TR103_2',
 'Дата и время',
 '34апп.сумма.5нитка',
 'Суммарная подача пара на 5 нитку, т/час',
 '34апп.удел.сумма.5нитка',
 'удел.сумм.5нитка',
 '120апп.удел.5нитка',
 'Массовая доля углеводородов, %',
 'Массовая доля тримеров, %']]
print(len(nit5.columns.to_list()), len(nit4.columns.to_list()))

In [ ]:
#depricated
dates = fin_df[['Дата и время']]
df = dates.merge(nit3_df, how = 'left', on = 'Дата и время')
df = df.merge(nit4_df, how = 'left', on = 'Дата и время')
df = df.merge(nit5_df, how = 'left', on = 'Дата и время')
df.info()
df = df.drop(columns = ['Массовая доля углеводородов, %_x','Массовая доля тримеров, %_x', 'Массовая доля углеводородов, %_y','Массовая доля тримеров, %_y',\
                  'Массовая доля углеводородов, %', 'Массовая доля тримеров, %'])
df = df.merge(fin_df[['Дата и время', 'Массовая доля углеводородов, %', 'Массовая доля тримеров, %']], how = 'left', on = 'Дата и время')
df.info()
#df.to_excel('final_clean_df.xlsx')
#pandas_profiling.ProfileReport(df = df, minimal=True).to_file("full_report(clean).html")